In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("Quiz2.ipynb")

# Quiz 2 — GroupBy, Pivot Tables, and Joining Tables

**Duration:** 50 minutes

This quiz uses three CSV files placed alongside the notebook:

- `orders.csv` — main table
- `returns.csv` — subset of `Order ID` with `Returned = Yes`
- `people.csv` — Region → Person mapping (RegionOwner)

> Q1 loads the CSVs. All subsequent questions must use the DataFrames created in Q1.


In [1]:
import pandas as pd
import numpy as np

#### Q1 — Load the CSVs
Load `orders.csv`, `returns.csv`, and `people.csv` into DataFrames named `orders`, `returns`, `people`.

- Parse `Order Date` and `Ship Date` as datetime where possible.
- Ensure `Sales`, `Profit`, `Quantity`, and `Discount` are numeric (coerce errors).


In [3]:

orders = pd.read_csv('orders.csv', parse_dates=['Order Date', 'Ship Date'])

num_cols = ["Sales", "Quantity", "Discount", "Profit"]

for col in num_cols:
  orders[col] = pd.to_numeric(orders[col], errors="coerce")

people = pd.read_csv('people.csv')
returns = pd.read_csv('returns.csv')
(len(orders), len(returns), len(people))



(12000, 840, 4)

In [ ]:
grader.check("Q1")

#### Q2 — Sales by Sub-Category (top 10)
Create `sales_by_subcat`: total Sales per `Sub-Category`, sorted descending, show `head(10)`.


In [5]:
sales_by_subcat = (
    orders
    .groupby("Sub-Category")['Sales']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

sales_by_subcat

Sub-Category
Phones         1486234.29
Copiers        1364767.02
Machines       1281804.92
Accessories    1245798.90
Chairs          857373.64
Bookcases       853681.65
Furnishings     813705.34
Tables          782093.65
Appliances      749318.64
Storage         732942.72
Name: Sales, dtype: float64

In [ ]:
grader.check("Q2")

#### Q3 — Region summary (multi-metric)
Create `region_summary` with index=`Region` and columns:
- `total_sales` (sum Sales)
- `avg_profit` (mean Profit)
- `order_count` (count of unique `Order ID`)


In [6]:
region_summary = (
    orders
    .groupby("Region")
    .agg(
        total_sales=("Sales", "sum"),
        avg_profit=("Profit", "mean"),
        order_count=("Order ID", "nunique")
    )
)

region_summary.head()

total_sales  avg_profit  order_count
Region                                       
Central   3360325.97  164.150163         3012
East      3457854.97  172.503548         3038
South     3265115.94  158.621872         2874
West      3581196.45  178.132058         3076

In [ ]:
grader.check("Q3")

#### Q4 — Monthly sales
Construct `monthly_sales` as a DataFrame with columns `['year_month','sales']` based on `Order Date` aggregated to year-month, sorted ascending; display `tail(12)`.


In [13]:
monthly_sales = (
    orders
    .assign(year_month=orders['Order Date'].dt.to_period("M"))
    .groupby('year_month')['Sales']
    .sum()
    .reset_index(name='sales')
    .sort_values("year_month")
)

monthly_sales.tail(12)

year_month      sales
36    2018-01  309526.09
37    2018-02  294872.25
38    2018-03  273444.57
39    2018-04  243006.22
40    2018-05  263965.21
41    2018-06  307755.28
42    2018-07  264502.20
43    2018-08  329236.52
44    2018-09  298704.98
45    2018-10  266227.01
46    2018-11  247585.84
47    2018-12  288797.77

In [ ]:
grader.check("Q4")

#### Q5 — Join Orders with Returns
Create `orders_ret` by left-joining `orders` with `returns[['Order ID','Returned']]` on `Order ID`. Missing → set `Returned='No'`.


In [22]:

orders_ret = pd.merge(
    orders,
    returns[['Order ID','Returned']],
    how='left',
    on='Order ID' )

orders_ret["Returned"] = orders_ret["Returned"].fillna("No")

orders_ret['Returned'].value_counts(dropna=False).head()

Returned
No     11160
Yes      840
Name: count, dtype: int64

In [ ]:
grader.check("Q5")

#### Q6 — Join Region Owner
Join `orders_ret` with `people` on `Region` to add `RegionOwner` (from `Person`).


In [24]:

orders_ret = pd.merge(
    orders_ret,
    people.rename(columns={"Person": "RegionOwner"}),
    on="Region",
    how="left"
)

orders_ret[['Region','RegionOwner']].drop_duplicates().head()

Region RegionOwner
0  Central        Sara
2     East      Howard
3    South       Kathy
7     West      Andrew

In [ ]:
grader.check("Q6")

#### Q7 — Pivot: Profit by Category × Region
Create `profit_cat_region` pivot with index=`Category`, columns=`Region`, values=`Profit`, `aggfunc='sum'`, and `fill_value=0`.


In [25]:

profit_cat_region = pd.pivot_table(
    orders,
    index="Category",
    columns="Region",
    values="Profit",
    aggfunc="sum",
    fill_value=0
)

profit_cat_region.head()

Region             Central       East      South       West
Category                                                   
Furniture        104983.14  107907.96  126179.92  143807.59
Office Supplies  197033.71  186268.71  164945.85  184471.74
Technology       192403.44  229889.11  164753.49  219654.88

In [ ]:
grader.check("Q7")

#### Q8 — Pivot: Average Discount by Segment × Ship Mode
Create `avg_discount_seg_ship` with index=`Segment`, columns=`Ship Mode`, values=`Discount`, `aggfunc='mean'`, `fill_value=0`.


In [27]:
avg_discount_seg_ship = pd.pivot_table(
    orders,
    index='Segment',
    columns='Ship Mode',
    values='Discount',
    aggfunc='mean',
    fill_value=0
)

avg_discount_seg_ship

Ship Mode    First Class  Same Day  Second Class  Standard Class
Segment                                                         
Consumer        0.111756  0.111360      0.103734        0.109364
Corporate       0.110273  0.093734      0.110425        0.107324
Home Office     0.112924  0.114835      0.098921        0.105207

In [ ]:
grader.check("Q8")

#### Q9 — Return rate by Region × Category
Engineer `is_returned = (Returned == 'Yes')`. Create `return_rate` pivot: mean of `is_returned` × 100, rounded(2), index=`Region`, columns=`Category`.


In [31]:
orders_ret_people["is_returned"] = (orders_ret_people["Returned"] == "Yes")
return_rate = (
    pd.pivot_table(
        orders_ret_people,
        index="Region",
        columns="Category",
        values="is_returned",
        aggfunc="mean"
    )
    * 100
).round(2)


return_rate.head()

Category  Furniture  Office Supplies  Technology
Region                                          
Central        6.67             7.40        6.49
East           5.99             7.11        6.24
South          6.92             6.90        8.28
West           7.08             7.22        7.51

In [ ]:
grader.check("Q9")

#### Q10 — Top loss-making products
Compute `top_losses` of 10 products with **lowest** total profit. Columns: `['Product Name','total_profit','order_count','return_rate']`, where `return_rate` = mean `is_returned` × 100 (round 1d).


In [34]:
orders_ret["is_returned"] = (orders_ret["Returned"] == "Yes")

top_losses = (
    orders_ret
    .groupby("Product Name")
    .agg(
        total_profit=("Profit", "sum"),
        order_count=("Order ID", "nunique"),
        return_rate=("is_returned", "mean")
    )
    .reset_index()
)

top_losses["return_rate"] = (top_losses["return_rate"] * 100).round(1)

top_losses = top_losses.sort_values("total_profit").head(10)

top_losses


Product Name  total_profit  order_count  return_rate
425           Eco Printer (Copiers)      -1774.72           11          9.1
295       Deluxe Desk (Furnishings)      -1245.73           16          6.2
326        Deluxe Paper (Envelopes)      -1206.55            9         11.1
829   Standard Copier (Accessories)      -1200.10            9         11.1
600   Executive Phone (Accessories)      -1158.74            6          0.0
381           Eco Copier (Machines)       -645.62            7         14.3
631    Portable Accessory (Copiers)       -418.81            3          0.0
705    Portable Stapler (Envelopes)       -400.95            5          0.0
27   Advanced Envelope (Appliances)       -351.02           14          7.1
14       Advanced Bookcase (Tables)       -347.75           14         21.4

In [ ]:
grader.check("Q10")

#### Q11 — First-order year × Segment (cohort-like)
For each `Customer ID`, compute `first_year` from earliest `Order Date`. Use that row’s `Segment` as `first_segment`. Build `cohort_seg` pivot: index=`first_year`, columns=`first_segment`, values = **number of unique customers**. Keep last 5 years. Fill 0.


In [40]:

first_orders= (
    orders
    .sort_values("Order Date")
    .groupby('Customer ID')
    .first()
    .reset_index()
)
first_orders["first_year"] = first_orders["Order Date"].dt.year



cohort_seg = pd.pivot_table(
    first_orders,
    index="first_year",
    columns="Segment",
    values="Customer ID",
    aggfunc="nunique",
    fill_value=0
)
cohort_seg = cohort_seg.sort_index().tail(5)

cohort_seg

Segment     Consumer  Corporate  Home Office
first_year                                  
2015            1466        677          366
2016            1120        502          275
2017             764        330          189
2018             541        207          167

In [ ]:
grader.check("Q11")

#### Q12 — RegionOwner dashboard
Per `RegionOwner` compute: `return_rate_owner` (mean `is_returned` × 100), `avg_profit_per_order` (sum `Profit` / nunique `Order ID`), and `orders` (nunique `Order ID`). Create `owner_dashboard` sorted by `return_rate_owner` descending.


In [42]:
owner_dashboard = (
    orders_ret_people
    .groupby("RegionOwner")
    .agg(
        return_rate_owner=("is_returned", "mean"),
        orders=("Order ID", "nunique"),
        total_profit=("Profit", "sum")
    )
)

owner_dashboard["return_rate_owner"] = (owner_dashboard["return_rate_owner"] * 100).round(2)

owner_dashboard["avg_profit_per_order"] = (
    owner_dashboard["total_profit"] / owner_dashboard["orders"]
)

owner_dashboard = owner_dashboard[
    ["return_rate_owner", "avg_profit_per_order", "orders"]
].sort_values("return_rate_owner", ascending=False)

owner_dashboard.head()


return_rate_owner  avg_profit_per_order  orders
RegionOwner                                                 
Andrew                    7.25            178.132058    3076
Kathy                     7.24            158.621872    2874
Sara                      6.97            164.150163    3012
Howard                    6.55            172.503548    3038

In [ ]:
grader.check("Q12")